In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


## Preprocessing Data

In [2]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

### Solving missing data

In [3]:
cabin_Nan = df_train['Cabin'].isna().sum()
cabin_Values = df_train['Cabin'].count()
print('% of nulls in Cabin Colums is: ' + str((cabin_Nan / (cabin_Values + cabin_Nan))*100) + '%')
df_train = df_train.drop(['Cabin'], axis = 1)
df_test = df_test.drop(['Cabin'], axis = 1)

% of nulls in Cabin Colums is: 77.10437710437711%


In [4]:
cabin_Nan = df_train['Embarked'].isna().sum()
cabin_Values = df_train['Embarked'].count()
print('% of nulls in Embarked Colums is: ' + str((cabin_Nan / (cabin_Values + cabin_Nan))*100) + '%')
df_train = df_train.dropna(subset=['Embarked'])
df_test = df_test.dropna(subset=['Embarked'])
#df_train = df_train.drop(['Embarked'], axis = 1)


% of nulls in Embarked Colums is: 0.22446689113355783%


In [5]:
df_train = df_train.drop(columns = ['Name', 'Ticket'], axis = 1)
X = df_train.iloc[:, 2: ].values
y = df_train.iloc[:, 1].values

df_test = df_test.drop(columns = ['Name', 'Ticket'], axis = 1)

X_final = df_test.iloc[:, 1:].values


### Encoding Categorical Data

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 2:3])
X[:, 2:3] = imputer.transform(X[:, 2:3])

In [7]:
imputer.fit(X_final[:, 2:3])
X_final[:, 2:3] = imputer.transform(X_final[:, 2:3])

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [7])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X_final = np.array(ct.fit_transform(X_final))
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [7])], remainder='passthrough')
X_final = np.array(ct.fit_transform(X_final))

### Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [11]:
X_final = sc.fit_transform(X_final)

### Spliting the data into training and test sets 

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Building the ANN

In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [14]:
classifier = Sequential()

### Adding the layers

In [15]:
classifier.add(Dense(output_dim = 5, init = 'uniform', activation = 'relu', input_dim = 10))
#In case of overfit:
#classifier.add(Dropout(p=0.1))
classifier.add(Dense(output_dim = 5, init = 'uniform', activation = 'relu'))

classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=10, units=5, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=5, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


### Compiling the ANN

In [16]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
classifier.fit(X_train, y_train, batch_size = 15, epochs = 100)

Epoch 1/100
711/711 [==============================] - 0s 625us/step - loss: 0.6905 - accuracy: 0.6188
Epoch 2/100
711/711 [==============================] - 0s 93us/step - loss: 0.6810 - accuracy: 0.6245
Epoch 3/100
711/711 [==============================] - 0s 95us/step - loss: 0.6513 - accuracy: 0.6512
Epoch 4/100
711/711 [==============================] - 0s 94us/step - loss: 0.5906 - accuracy: 0.7975
Epoch 5/100
711/711 [==============================] - 0s 95us/step - loss: 0.5245 - accuracy: 0.8242
Epoch 6/100
711/711 [==============================] - 0s 93us/step - loss: 0.4778 - accuracy: 0.8256
Epoch 7/100
711/711 [==============================] - 0s 96us/step - loss: 0.4490 - accuracy: 0.8242
Epoch 8/100
711/711 [==============================] - 0s 91us/step - loss: 0.4322 - accuracy: 0.8270
Epoch 9/100
711/711 [==============================] - 0s 94us/step - loss: 0.4207 - accuracy: 0.8256
Epoch 10/100
711/711 [==============================] - 0s 96us/step - loss: 0.41

711/711 [==============================] - 0s 95us/step - loss: 0.3688 - accuracy: 0.8453
Epoch 81/100
711/711 [==============================] - 0s 93us/step - loss: 0.3686 - accuracy: 0.8467
Epoch 82/100
711/711 [==============================] - 0s 93us/step - loss: 0.3686 - accuracy: 0.8495
Epoch 83/100
711/711 [==============================] - 0s 95us/step - loss: 0.3687 - accuracy: 0.8495
Epoch 84/100
711/711 [==============================] - 0s 95us/step - loss: 0.3673 - accuracy: 0.8467
Epoch 85/100
711/711 [==============================] - 0s 93us/step - loss: 0.3678 - accuracy: 0.8467
Epoch 86/100
711/711 [==============================] - 0s 93us/step - loss: 0.3671 - accuracy: 0.8467
Epoch 87/100
711/711 [==============================] - 0s 96us/step - loss: 0.3672 - accuracy: 0.8481
Epoch 88/100
711/711 [==============================] - 0s 94us/step - loss: 0.3663 - accuracy: 0.8509
Epoch 89/100
711/711 [==============================] - 0s 93us/step - loss: 0.3664 - 

### Getting the results

In [18]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred = [int(elem) for elem in y_pred]

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [20]:
def calculate_accuracy():
    right = cm[0][0] + cm[1,1]
    total = cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1] 
    print(str((right/total) * 100) + '%')

In [21]:
calculate_accuracy()

73.59550561797754%


## Submit to leaderboard

In [22]:
y_pred = classifier.predict(X_final)
y_pred = (y_pred > 0.5)
y_pred = [int(elem) for elem in y_pred]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


In [23]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': y_pred})
output.to_csv('my_submission_ANN_2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


## Evaluating the ANN

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [25]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [26]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print('The mean accuracy is:' + str(mean) + ' and the variance is: ' + str(variance))

The mean accuracy is:0.6245305061340332 and the variance is: 0.04118515030559009


### Tunning the ANN

In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [28]:
def build_classifier(optimizer, output_dim):
    classifier = Sequential()
    classifier.add(Dense(output_dim = output_dim, init = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(output_dim = output_dim, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [29]:
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size' : [15,27],
              'nb_epoch' : [100,300],
              'optimizer' : ['adam', 'rmsprop'],
              'output_dim' : [5,6]}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=10, units=5, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=5, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  """


Epoch 1/1
639/639 [==============================] - 0s 345us/step - loss: 0.6905 - accuracy: 0.6291
Epoch 1/1
640/640 [==============================] - 0s 341us/step - loss: 0.6910 - accuracy: 0.6078
Epoch 1/1
640/640 [==============================] - 0s 328us/step - loss: 0.6901 - accuracy: 0.6359
Epoch 1/1
640/640 [==============================] - 0s 334us/step - loss: 0.6913 - accuracy: 0.6078
Epoch 1/1
640/640 [==============================] - 0s 327us/step - loss: 0.6908 - accuracy: 0.6250
Epoch 1/1
640/640 [==============================] - 0s 326us/step - loss: 0.6916 - accuracy: 0.6094
Epoch 1/1
640/640 [==============================] - 0s 325us/step - loss: 0.6911 - accuracy: 0.6203
Epoch 1/1
640/640 [==============================] - 0s 322us/step - loss: 0.6912 - accuracy: 0.6203
Epoch 1/1
640/640 [==============================] - 0s 317us/step - loss: 0.6912 - accuracy: 0.6125
Epoch 1/1
640/640 [==============================] - 0s 314us/step - loss: 0.6909 - accurac

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=10, units=6, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  after removing the cwd from sys.path.


Epoch 1/1
639/639 [==============================] - 0s 320us/step - loss: 0.6912 - accuracy: 0.6166
Epoch 1/1
640/640 [==============================] - 0s 314us/step - loss: 0.6910 - accuracy: 0.6187
Epoch 1/1
640/640 [==============================] - 0s 319us/step - loss: 0.6908 - accuracy: 0.6297
Epoch 1/1
640/640 [==============================] - 0s 324us/step - loss: 0.6907 - accuracy: 0.6156
Epoch 1/1
640/640 [==============================] - 0s 328us/step - loss: 0.6909 - accuracy: 0.6219
Epoch 1/1
640/640 [==============================] - 0s 375us/step - loss: 0.6911 - accuracy: 0.6203
Epoch 1/1
640/640 [==============================] - 0s 338us/step - loss: 0.6909 - accuracy: 0.6109
Epoch 1/1
640/640 [==============================] - 0s 341us/step - loss: 0.6914 - accuracy: 0.6141
Epoch 1/1
640/640 [==============================] - 0s 338us/step - loss: 0.6906 - accuracy: 0.6281
Epoch 1/1
640/640 [==============================] - 0s 333us/step - loss: 0.6901 - accurac

640/640 [==============================] - 0s 280us/step - loss: 0.6918 - accuracy: 0.6187
Epoch 1/1
640/640 [==============================] - 0s 276us/step - loss: 0.6921 - accuracy: 0.6219
Epoch 1/1
640/640 [==============================] - 0s 276us/step - loss: 0.6920 - accuracy: 0.6078
Epoch 1/1
640/640 [==============================] - 0s 288us/step - loss: 0.6918 - accuracy: 0.6125
Epoch 1/1
640/640 [==============================] - 0s 290us/step - loss: 0.6920 - accuracy: 0.6125
Epoch 1/1
640/640 [==============================] - 0s 280us/step - loss: 0.6917 - accuracy: 0.6313
Epoch 1/1
640/640 [==============================] - 0s 279us/step - loss: 0.6920 - accuracy: 0.6266
Epoch 1/1
640/640 [==============================] - 0s 284us/step - loss: 0.6917 - accuracy: 0.6203
Epoch 1/1
640/640 [==============================] - 0s 281us/step - loss: 0.6918 - accuracy: 0.6234
Epoch 1/1
639/639 [==============================] - 0s 255us/step - loss: 0.6908 - accuracy: 0.6213


In [30]:
best_accuracy
best_parameters

{'batch_size': 15, 'nb_epoch': 100, 'optimizer': 'rmsprop', 'output_dim': 6}